# LSTM Stock Prediction Models

### Import Required Libraries

In [476]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import hvplot.pandas
%matplotlib inline

### Import Scaler

In [477]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

## Functions to Load in DataFrames by List Index or Ticker

In [488]:
# Read in list of csv's
csv_path = 'Daily_Stocks_Metrics/csv_names_list.csv'
csv_df = pd.read_csv(csv_path)
csv_df.columns = ['csv_names']
csv_list = list(csv_df.csv_names.values)
csv_list[:4]

['Daily_Stocks_Metrics/ILMN UW Equity.csv',
 'Daily_Stocks_Metrics/EIX UN Equity.csv',
 'Daily_Stocks_Metrics/GPS UN Equity.csv',
 'Daily_Stocks_Metrics/ES UN Equity.csv']

In [489]:
# splice csv name to retrieve tickers
csv_df['tickers'] = csv_df.csv_names.str[21:-13]
csv_df['tickers'] = csv_df['tickers'].str.replace(' ','')
csv_df.head()

,csv_names,tickers
0,Daily_Stocks_Metrics/ILMN UW Equity.csv,ILMN
1,Daily_Stocks_Metrics/EIX UN Equity.csv,EIX
2,Daily_Stocks_Metrics/GPS UN Equity.csv,GPS
3,Daily_Stocks_Metrics/ES UN Equity.csv,ES
4,Daily_Stocks_Metrics/NLOK UW Equity.csv,NLOK


In [490]:
# turn dataframe into a dictionary for fastest lookup
csv_df.index = csv_df.csv_names
csv_df.drop(columns=['csv_names'], inplace=True)
csv_dict = csv_df.T.to_dict('list')

In [481]:
# csv_dict = dict([(key,value[0]) for key,value in csv_dict.items()])

In [494]:
# test the dictionary
print(f'Testing the dictionary: {csv_dict[csv_list[137]]}')
print(f'Length of Dictionary: {len(csv_dict)}')

Testing the dictionary: ['HP']
Length of Dictionary: 505


In [496]:
# define new dictionary to swap key, value pairs
ticker_dict = dict([(value[0],key) for key,value in csv_dict.items()])

In [497]:
# test the dictionary
print(ticker_dict['AAPL'])

Daily_Stocks_Metrics/AAPL UW Equity.csv


### **Function to Load DataFrame by Ticker**

In [183]:
# Define Columns to call on Dataframe, this excludes Fundamental Data for the time being
cols = ['px', 'Price Momentum 12M', 'Price Momentum 12M-1M',
       'Price to 52W High', 'Price to 52W Low', 'Return 1D', 'Return 1M',
       'Return 1Q', 'Max Daily Return over 1M', 'Realized Volatility 1M',
       'Realized Volatility 3M']

In [184]:
# outputs a dataframe based on input ticker, checks to make sure price column not null

def load_df_by_ticker(ticker):
    stock_df = pd.read_csv(ticker_dict[ticker])
    stock_df = stock_df[cols]
    if 'px' in stock_df.columns:
        print(f'{str(ticker)} DataFrame created')
        return stock_df    
    else:
        print('error')
    

In [499]:
# test the function for Google
load_df_by_ticker('GOOG').head()

GOOG DataFrame created


,px,Price Momentum 12M,Price Momentum 12M-1M,Price to 52W High,Price to 52W Low,Return 1D,Return 1M,Return 1Q,Max Daily Return over 1M,Realized Volatility 1M,Realized Volatility 3M
0,552.991,-0.006452,-0.016305,0.930260,1.125794,0.013136,0.010016,0.098279,0.023172,0.012255,0.013539
1,549.332,-0.016356,-0.019623,0.924104,1.118345,-0.006617,0.003333,0.077756,0.023172,0.012354,0.013507
2,557.968,0.004669,-0.025248,0.938632,1.135926,0.015721,0.030692,0.083568,0.023172,0.012502,0.013583
3,556.462,-0.016879,-0.049105,0.936099,1.132860,-0.002699,0.033889,0.063101,0.023172,0.012430,0.013452
4,558.825,-0.011711,-0.042557,0.940074,1.137671,0.004246,0.032217,0.056106,0.023172,0.012408,0.013400


### **Function to Load DataFrame by List Index**

In [186]:
# loads dataframe based on index value, input between [0,504]
def load_df_by_number(i):
    stock_df = pd.read_csv(csv_list[i])
    stock_df = stock_df[cols]
    if 'px' in stock_df.columns:
        print(f'{str(csv_dict[csv_list[i]])} DataFrame created')
        return stock_df    
    else:
        print('error')
    

In [502]:
# test the function
load_df_by_number(340).head()

['NTAP'] DataFrame created


,px,Price Momentum 12M,Price Momentum 12M-1M,Price to 52W High,Price to 52W Low,Return 1D,Return 1M,Return 1Q,Max Daily Return over 1M,Realized Volatility 1M,Realized Volatility 3M
0,8.6405,1.156459,1.659878,0.624011,2.280537,-0.067656,-0.189264,-0.264507,0.102303,0.043831,0.048269
1,9.5945,1.315778,1.835534,0.692909,2.532332,0.110410,-0.183301,-0.187072,0.110410,0.044884,0.050359
2,9.8126,1.311403,1.593904,0.708660,2.589897,0.022732,-0.108909,-0.164736,0.110410,0.043597,0.050454
3,10.0307,1.397395,1.501625,0.724411,2.647461,0.022227,-0.041665,-0.058822,0.110410,0.042702,0.049165
4,10.1669,1.422075,1.415595,0.734247,2.683409,0.013578,0.002683,-0.092460,0.110410,0.042241,0.048764


## **Prepare Data for LSTM**

In [504]:
# function to return X_train, X_test, y_train, y_test given a dataframe, lookback size, offset days and training fraction as float

## This took extremely long to write ;)

def structure_data(df, lookback_size, offset_days, split_float):
    '''This function will turn a dataframe for a stock into X_train,X_test,y_train,y_test.'''
    split = int(split_float*len(df))
    X = []
    y = []
    scaled_df = pd.DataFrame(scaler.fit_transform(df))
    prices = df.px.to_frame()
    price_scaler = scaler.fit(prices)
    print('Data Scaled')
    print(f'{scaled_df.head(1)}')
#     prices = df.px.to_frame()
#     price_scaler = scaler.fit_transform(prices)
    sample_size = int(len(scaled_df) - lookback_size - offset_days)
    for i in range(sample_size):
        features = scaled_df.iloc[i:i+lookback_size,:].to_numpy()
        target = scaled_df.iloc[i+lookback_size+offset_days,0]
        X.append(features)
        y.append(target)
        
    X_train = X[:split]
    print('X_train created')
    X_test = X[split:]
    print('X_test created')
    y_train = y[:split]
    print('y_train created')
    y_test = y[split:]
    print('y_test created')

#    return X_train, X_test, y_train, y_test
    return np.array(X_train), np.array(X_test), np.array(y_train).reshape(-1,1), np.array(y_test).reshape(-1,1)

In [505]:
# test the function 
a, b, c, d = structure_data(load_df_by_ticker('GOOG'), 12, 15, .7)

GOOG DataFrame created
Data Scaled
          0         1         2        3         4         5         6  \
0  0.043682  0.167213  0.200405  0.69719  0.216504  0.379393  0.389179   

          7         8         9        10  
0  0.582375  0.124923  0.260026  0.310266  
X_train created
X_test created
y_train created
y_test created


In [508]:
# see the structure of outputs
print(a.shape, b.shape, c.shape, d.shape)

(865, 12, 11) (345, 12, 11) (865, 1) (345, 1)


## Run LSTM

In [339]:
# specify units and dropout
number_units = 10
drop_out=.3

In [452]:
# define rnn model

rnn = Sequential()
rnn.add(LSTM(units=number_units, return_sequences=True, batch_input_shape=(2,12,11)))
rnn.add(Dropout(drop_out))
rnn.add(LSTM(units=number_units))
rnn.add(Dropout(drop_out))
rnn.add(Dense(1))
rnn.compile(optimizer='adam',  loss= 'mean_squared_error')
        
rnn.fit(a, c, epochs=100, shuffle=False, batch_size=2, verbose=1)

Train on 865 samples
Epoch 1/100
865/865 [==============================] - 12s 14ms/sample - loss: 0.0070
Epoch 2/100
865/865 [==============================] - 7s 8ms/sample - loss: 0.0157
Epoch 3/100
865/865 [==============================] - 7s 8ms/sample - loss: 0.0183
Epoch 4/100
865/865 [==============================] - 7s 9ms/sample - loss: 0.0189
Epoch 5/100
865/865 [==============================] - 7s 8ms/sample - loss: 0.0141
Epoch 6/100
865/865 [==============================] - 7s 8ms/sample - loss: 0.0099
Epoch 7/100
865/865 [==============================] - 7s 8ms/sample - loss: 0.0080
Epoch 8/100
865/865 [==============================] - 7s 9ms/sample - loss: 0.0075
Epoch 9/100
865/865 [==============================] - 7s 8ms/sample - loss: 0.0068
Epoch 10/100
865/865 [==============================] - 6s 7ms/sample - loss: 0.0064 0s
Epoch 11/100
865/865 [==============================] - 7s 8ms/sample - loss: 0.0059
Epoch 12/100
865/865 [==========================

In [453]:
rnn.evaluate(b,d)

345/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.013591585360928297

## Inspect 1st run results

In [509]:
# get scaler to reverse the scaling
new_scaler = MinMaxScaler()
prices = load_df_by_ticker('GOOG').px.to_frame()
new_scaler = new_scaler.fit(prices)

GOOG DataFrame created


In [455]:
# write results into a dataframe

predicted = rnn.predict(b)
predicted_prices = new_scaler.inverse_transform(predicted)
real_prices = new_scaler.inverse_transform(d.reshape(-1, 1))

stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
})
stocks.head()



stocks['Percentage Error']= stocks.Predicted/stocks.Real-1
stocks.head()

,Real,Predicted,Percentage Error
0,1214.38,1181.262085,-0.027271
1,1206.49,1171.490723,-0.029009
2,1200.96,1159.914551,-0.034177
3,1207.77,1149.477905,-0.048264
4,1201.62,1143.062500,-0.048732


In [466]:
# calculate some metrics
stocks['APE'] = abs(stocks.Predicted/stocks.Real-1) 
mape = stocks['APE'].mean()
max_down = stocks['Percentage Error'].min()
max_up = stocks['Percentage Error'].max()

In [467]:
print(mape, mpe, max_down, max_up)

0.06908084294092466 0.027230136236806188 -0.12136452557763111 0.21792504651136424


In [473]:
stocks.iloc[:,:2].hvplot(title = 'Google Stock Prices December 2018-November 2019')

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [469]:
# see percentage error as a funciton of time
stocks.iloc[:,2].hvplot()

:Curve   [index]   (Percentage Error)

In [3]:
# LSTM for hyperparameter tuning, did not produce significant results in the allotted time

# def run_2_LSTM(number_units, dropout, X_train, y_train, optimizer, loss, epoch, X_test, y_test):
#     rnn = Sequential()
#     rnn.add(LSTM(units=number_units, return_sequences=True, input_shape=(X_array.shape[1],1)))
#     rnn.add(Dropout(drop_out))
#     rnn.add(LSTM(units=number_units, return_sequences=True))
#     rnn.add(Dropout(drop_out))
# #     rnn.add(LSTM(units=number_units, return_sequences=True))
# #     rnn.add(Dropout(drop_out))
#     rnn.add(Dense(1))
#     rnn.compile(optimizer=optimizer, loss=loss)
#     rnn.fit(X_train, y_train, epochs=epoch, shuffle=False, batch_size=1, verbose=0)
#     loss = rnn.evaluate(x=X_test,y=y_test)
#     return loss

### Predicting the S&P Index

In [384]:
sp500_df = pd.read_csv('Daily_Stocks_Metrics/sp500.csv',index_col=['Date'],infer_datetime_format=True)

sp500_df.head(2)

,px,Price Momentum 12M,Price Momentum 12M-1M,Price to 52W High,Price to 52W Low,Return 1D,Return 1M,Return 1Q,Max Daily Return over 1M,Realized Volatility 1M,Realized Volatility 3M
Date,,,,,,,,,,,
1994-01-03,567.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994-01-04,569.07,NaN,NaN,NaN,NaN,0.003492,NaN,NaN,NaN,NaN,NaN


In [383]:
sp500_df.isnull().count()

px                          6767
Price Momentum 12M          6767
Price Momentum 12M-1M       6767
Price to 52W High           6767
Price to 52W Low            6767
Return 1D                   6767
Return 1M                   6767
Return 1Q                   6767
Max Daily Return over 1M    6767
Realized Volatility 1M      6767
Realized Volatility 3M      6767
dtype: int64

In [385]:
sp500_df.dropna(axis=1, how='all', inplace=True)

In [388]:
sp500_df.dropna(inplace=True)

In [ ]:
a,b,c,d,new_scaler=structure

In [470]:
a, b, c, d, new_scaler= structure_data(sp500_df, 12, 15, .7)

Data Scaled
          0         1         2         3         4         5         6  \
0  0.000000  0.507682  0.517671  0.438165  0.607432  0.684501  0.037189   
1  0.000531  0.507803  0.516309  0.463306  0.610188  0.689758  0.037189   
2  0.001808  0.520105  0.514974  0.498483  0.635413  0.700666  0.063427   
3  0.002400  0.531609  0.521070  0.465690  0.645083  0.710889  0.063427   
4  0.002340  0.536829  0.527215  0.435192  0.643074  0.713523  0.063427   

          7  
0  0.027985  
1  0.029198  
2  0.042477  
3  0.043728  
4  0.044216  
X_train created
X_test created
y_train created
y_test created


In [472]:
rnn = Sequential()
rnn.add(LSTM(units=number_units, return_sequences=True, batch_input_shape=(2,12,a.shape[2])))
rnn.add(Dropout(drop_out))
rnn.add(LSTM(units=number_units))
rnn.add(Dropout(drop_out))
rnn.add(Dense(1))
rnn.compile(optimizer='adam',  loss= 'mean_squared_error')
        
rnn.fit(a, c, epochs=10, shuffle=False, batch_size=2, verbose=1)

Train on 1491 samples
Epoch 1/10
1491/1491 [==============================] - 17s 11ms/sample - loss: 0.0014
Epoch 2/10
1491/1491 [==============================] - 10s 7ms/sample - loss: 0.0017
Epoch 3/10
1491/1491 [==============================] - 10s 7ms/sample - loss: 0.0021
Epoch 4/10
1491/1491 [==============================] - 10s 7ms/sample - loss: 0.0025
Epoch 5/10
1491/1491 [==============================] - 10s 7ms/sample - loss: 0.0033
Epoch 6/10
1491/1491 [==============================] - 10s 7ms/sample - loss: 0.0033
Epoch 7/10
1491/1491 [==============================] - 10s 7ms/sample - loss: 0.0032
Epoch 8/10
1491/1491 [==============================] - 10s 6ms/sample - loss: 0.0031
Epoch 9/10
1491/1491 [==============================] - 10s 6ms/sample - loss: 0.0028
Epoch 10/10
1491/1491 [==============================] - 10s 6ms/sample - loss: 0.0024


In [474]:
rnn.evaluate(b,d)

612/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.13948299507452616

S&P model clearly overfit, likely can't be fixed by tuning. Input features will have to be changed. 